<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/5_flower_dataset_Neural_Network_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
# Sweep config for selecting experiment
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'hidden_nodes': {'values': [128]},
        'img_size': {'values': [16]},
        'epochs': {'values': [10]},
        'experiment': {'values': ['dropout_only', 'batchnorm_only', 'full']}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-experiments")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: d0jbszm4
Sweep URL: https://wandb.ai/praveenkumar-aiml27-jntuh-university/5-flowers-experiments/sweeps/d0jbszm4


In [3]:
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        model = keras.Sequential()
        model.add(keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)))

        if config.experiment == 'batchnorm_only':
            model.add(keras.layers.Dense(config.hidden_nodes, use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))

        elif config.experiment == 'dropout_only':
            model.add(keras.layers.Dense(config.hidden_nodes, activation="relu"))
            model.add(keras.layers.Dropout(0.5))

        elif config.experiment == 'full':
            model.add(keras.layers.Dense(config.hidden_nodes,
                                          kernel_regularizer=keras.regularizers.l2(0.01),
                                          use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))
            model.add(keras.layers.Dropout(0.5))

        model.add(keras.layers.Dense(len(CLASS_NAMES), activation="softmax"))

        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]
        if config.experiment == 'full':
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True))

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6a7xlhxl with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: dropout_only
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: praveenkumar-aiml27 (praveenkumar-aiml27-jntuh-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
    413/Unknown 302s 725ms/step - accuracy: 0.2597 - loss: 1.6341

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 336s 806ms/step - accuracy: 0.2597 - loss: 1.6340 - val_accuracy: 0.3838 - val_loss: 1.4387
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 261s 631ms/step - accuracy: 0.3770 - loss: 1.4498 - val_accuracy: 0.4000 - val_loss: 1.3514
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 216s 521ms/step - accuracy: 0.4209 - loss: 1.3861 - val_accuracy: 0.4324 - val_loss: 1.3012
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 230s 443ms/step - accuracy: 0.4219 - loss: 1.3317 - val_accuracy: 0.4541 - val_loss: 1.2736
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 441ms/step - accuracy: 0.4425 - loss: 1.3270 - val_accuracy: 0.4649 - val_loss: 1.2418
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 439ms/step - accuracy: 0.4710 - loss: 1.2798 - val_accuracy: 0.4784 - val_loss: 1.2325
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 154s 373ms/step - accuracy: 0.4762 - loss: 1.2548 - val_accuracy: 0.4811 - val_loss: 1.2182
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 373ms/step - accuracy: 0.4806 - loss: 1.23

batch/accuracy,▁▁▃▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▅▅▆▆▇▇█▇▇▇▇▇▇▇▆▇█▇▇███
batch/batch_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▇▇▆▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁
epoch/accuracy,▁▄▅▆▆▇█▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▃▃▃▂▂▁▁
epoch/val_accuracy,▁▂▄▅▆▇▇▇▇█
epoch/val_loss,█▆▄▃▂▂▂▁▁▁
batch/accuracy,0.49605


wandb: Agent Starting Run: iw6stuoo with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: batchnorm_only
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


Epoch 1/10
     70/Unknown 21s 285ms/step - accuracy: 0.2383 - loss: 1.8100